In [116]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

En este script procesare los datos casteando a sus tipos originales, eliminando atipicos. 
La prediccion la realizare con una regreson logistica (si no supera la marca del la baseline) se realizara con un
modelo superior.

In [2]:
df_train = pd.read_csv('train.csv', index_col='id')
df_train

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
id,,,,,,,,,,,,,,,,,
0,Male,24.443011,1.699998,81.669950,yes,yes,2.000000,2.983297,Sometimes,no,2.763573,no,0.000000,0.976473,Sometimes,Public_Transportation,Overweight_Level_II
1,Female,18.000000,1.560000,57.000000,yes,yes,2.000000,3.000000,Frequently,no,2.000000,no,1.000000,1.000000,no,Automobile,Normal_Weight
2,Female,18.000000,1.711460,50.165754,yes,yes,1.880534,1.411685,Sometimes,no,1.910378,no,0.866045,1.673584,no,Public_Transportation,Insufficient_Weight
3,Female,20.952737,1.710730,131.274851,yes,yes,3.000000,3.000000,Sometimes,no,1.674061,no,1.467863,0.780199,Sometimes,Public_Transportation,Obesity_Type_III
4,Male,31.641081,1.914186,93.798055,yes,yes,2.679664,1.971472,Sometimes,no,1.979848,no,1.967973,0.931721,Sometimes,Public_Transportation,Overweight_Level_II
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20753,Male,25.137087,1.766626,114.187096,yes,yes,2.919584,3.000000,Sometimes,no,2.151809,no,1.330519,0.196680,Sometimes,Public_Transportation,Obesity_Type_II
20754,Male,18.000000,1.710000,50.000000,no,yes,3.000000,4.000000,Frequently,no,1.000000,no,2.000000,1.000000,Sometimes,Public_Transportation,Insufficient_Weight
20755,Male,20.101026,1.819557,105.580491,yes,yes,2.407817,3.000000,Sometimes,no,2.000000,no,1.158040,1.198439,no,Public_Transportation,Obesity_Type_II


In [3]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20758 entries, 0 to 20757
Data columns (total 17 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Gender                          20758 non-null  object 
 1   Age                             20758 non-null  float64
 2   Height                          20758 non-null  float64
 3   Weight                          20758 non-null  float64
 4   family_history_with_overweight  20758 non-null  object 
 5   FAVC                            20758 non-null  object 
 6   FCVC                            20758 non-null  float64
 7   NCP                             20758 non-null  float64
 8   CAEC                            20758 non-null  object 
 9   SMOKE                           20758 non-null  object 
 10  CH2O                            20758 non-null  float64
 11  SCC                             20758 non-null  object 
 12  FAF                             20758

In [10]:
# Castear variables float a enteras
def feature_float_int (df):
    df_copy = df.copy()
    columns_to_transform = ['Age', 'FCVC', 'NCP', 'CH2O', 'FAF', 'TUE']

    for i in columns_to_transform:
        df_copy[i]=df_copy[i].astype(int)
    
    print('Casteo realizado con exito')
    return df_copy

In [11]:
df_train = feature_float_int(df_train)
df_train

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
id,,,,,,,,,,,,,,,,,
0,Male,24,1.699998,81.669950,yes,yes,2,2,Sometimes,no,2,no,0,0,Sometimes,Public_Transportation,Overweight_Level_II
1,Female,18,1.560000,57.000000,yes,yes,2,3,Frequently,no,2,no,1,1,no,Automobile,Normal_Weight
2,Female,18,1.711460,50.165754,yes,yes,1,1,Sometimes,no,1,no,0,1,no,Public_Transportation,Insufficient_Weight
3,Female,20,1.710730,131.274851,yes,yes,3,3,Sometimes,no,1,no,1,0,Sometimes,Public_Transportation,Obesity_Type_III
4,Male,31,1.914186,93.798055,yes,yes,2,1,Sometimes,no,1,no,1,0,Sometimes,Public_Transportation,Overweight_Level_II
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20753,Male,25,1.766626,114.187096,yes,yes,2,3,Sometimes,no,2,no,1,0,Sometimes,Public_Transportation,Obesity_Type_II
20754,Male,18,1.710000,50.000000,no,yes,3,4,Frequently,no,1,no,2,1,Sometimes,Public_Transportation,Insufficient_Weight
20755,Male,20,1.819557,105.580491,yes,yes,2,3,Sometimes,no,2,no,1,1,no,Public_Transportation,Obesity_Type_II


In [23]:
# Eliminar valores atipicos de Age y Height

def delete_outliers (df):
    df_copy = df.copy()
    out=[]

    print('Tamaño del df antes de remover outliers:',df_copy.shape)
    columns_outliers = ['Age', 'Height']

    for i in columns_outliers:
        q1 = df_copy[i].quantile(0.25)
        q3 = df_copy[i].quantile(0.75)
        iqr = q3-q1
        Lower_tail = q1 - 1.5 * iqr
        Upper_tail = q3 + 1.5 * iqr
    
        outliers = df_copy[(df_copy[i] < Lower_tail) | (df_copy[i] > Upper_tail)]
        out.extend(outliers.index)

    out = list(set(out))
    df_copy = df_copy.drop(out)
    print("Outliers removidos. Tamaño del nuevo df:", df_copy.shape)
    return df_copy

In [24]:
df_train= delete_outliers (df_train)

Tamaó del df antes de remover outliers: (20758, 17)
Outliers removidos. Tamaño del nuevo df: (19725, 17)


In [26]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19725 entries, 0 to 20757
Data columns (total 17 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Gender                          19725 non-null  object 
 1   Age                             19725 non-null  int32  
 2   Height                          19725 non-null  float64
 3   Weight                          19725 non-null  float64
 4   family_history_with_overweight  19725 non-null  object 
 5   FAVC                            19725 non-null  object 
 6   FCVC                            19725 non-null  int32  
 7   NCP                             19725 non-null  int32  
 8   CAEC                            19725 non-null  object 
 9   SMOKE                           19725 non-null  object 
 10  CH2O                            19725 non-null  int32  
 11  SCC                             19725 non-null  object 
 12  FAF                             19725

In [42]:
# Separamos en X e y
X = df_train.iloc[:, :-1]
y = df_train.iloc[:, 16]

In [44]:
# Codificar valores categoricos
def dummies_variable (X):
    X_copy = X.copy()
    categorical_var = X_copy.select_dtypes(include='object')

    # Creamos instancia de encoder
    encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
    encoder.fit(categorical_var)
    # Codificamos
    categorical_encoded = encoder.transform(categorical_var)

    # Convertimos a df
    X_copy = pd.DataFrame(categorical_encoded, columns=encoder.get_feature_names_out(categorical_var.columns))

    required_columns = X_copy.columns

    # Comprobamos si todas las columnas existen en X_copy, sea que estan o no, se retornaran
    if all(column in X_copy.columns for column in required_columns):
        return X_copy   
    return X_copy
    

In [45]:
X_categorical = dummies_variable (X)

In [54]:
X_categorical.head()

,Gender_Female,Gender_Male,family_history_with_overweight_no,family_history_with_overweight_yes,FAVC_no,FAVC_yes,CAEC_Always,CAEC_Frequently,CAEC_Sometimes,CAEC_no,...,SCC_no,SCC_yes,CALC_Frequently,CALC_Sometimes,CALC_no,MTRANS_Automobile,MTRANS_Bike,MTRANS_Motorbike,MTRANS_Public_Transportation,MTRANS_Walking
0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
3,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


In [50]:
# Normalizar los datos nuemericos
def normalizer_numerical (X):
    X_copy = X.copy()
    numerical_val = X_copy.select_dtypes(include='number')
    columns_name = numerical_val.columns
    # escalado y transformacion
    scaler = StandardScaler()
    nuemrical_scaled = scaler.fit_transform(numerical_val)

    # Convertimos a df
    numerical_df = pd.DataFrame(nuemrical_scaled, columns=columns_name)

    return numerical_df


In [51]:
X_numerical = normalizer_numerical (X)

In [55]:
X_numerical.head()

,Age,Height,Weight,FCVC,NCP,CH2O,FAF,TUE
0,0.309474,-0.035401,-0.236260,-0.519906,-0.893535,0.365503,-0.952392,-0.700633
1,-1.127807,-1.665248,-1.157023,-0.519906,0.431552,0.365503,0.256095,0.973705
2,-1.127807,0.098038,-1.412100,-2.196680,-2.218622,-1.405886,-0.952392,0.973705
3,-0.648713,0.089540,1.615156,1.156868,0.431552,-1.405886,0.256095,-0.700633
4,1.986301,2.458160,0.216400,-0.519906,-2.218622,-1.405886,0.256095,-0.700633


In [53]:
# Funcion para concatenar categoricas y numericas
def concat_types(num,cat):
    X_ = pd.concat([num,cat], axis=1)
    return X_

In [56]:
X = concat_types(X_categorical,X_numerical)
X

,Gender_Female,Gender_Male,family_history_with_overweight_no,family_history_with_overweight_yes,FAVC_no,FAVC_yes,CAEC_Always,CAEC_Frequently,CAEC_Sometimes,CAEC_no,...,MTRANS_Public_Transportation,MTRANS_Walking,Age,Height,Weight,FCVC,NCP,CH2O,FAF,TUE
0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.309474,-0.035401,-0.236260,-0.519906,-0.893535,0.365503,-0.952392,-0.700633
1,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,-1.127807,-1.665248,-1.157023,-0.519906,0.431552,0.365503,0.256095,0.973705
2,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,...,1.0,0.0,-1.127807,0.098038,-1.412100,-2.196680,-2.218622,-1.405886,-0.952392,0.973705
3,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,...,1.0,0.0,-0.648713,0.089540,1.615156,1.156868,0.431552,-1.405886,0.256095,-0.700633
4,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,...,1.0,0.0,1.986301,2.458160,0.216400,-0.519906,-2.218622,-1.405886,0.256095,-0.700633
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19720,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.549020,0.740277,0.977385,-0.519906,0.431552,0.365503,0.256095,-0.700633
19721,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,1.0,0.0,-1.127807,0.081041,-1.418286,1.156868,1.756638,-1.405886,1.464582,0.973705
19722,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,...,1.0,0.0,-0.648713,1.356496,0.656159,-0.519906,0.431552,0.365503,0.256095,0.973705
19723,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,2.465394,-0.035378,-0.167206,-0.519906,-2.218622,0.365503,-0.952392,-0.700633


In [57]:
# Entrenar modelo y mostrar resultados entrenamiento y validacion
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=123, stratify=y)

In [59]:
X_train.shape

(15780, 30)

In [64]:
y_train.shape

(15780,)

VAMOS A REALIZAR VARIAS PREDICCIONES CON 3 ALGORITMOS

In [95]:
# Regresion logistica
model_v1 = LogisticRegression(random_state=123, multi_class='multinomial', n_jobs=-1, verbose=1)

In [96]:
model_v1.fit(X_train, y_train)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


LogisticRegression(multi_class='multinomial', n_jobs=-1, random_state=123,
                   verbose=1)

El modelo de LR obtiene un 87% de accuracy, un 1% mas que en la baseline.

In [112]:
# KNeighborsClassifier
model_v2 = KNeighborsClassifier(n_neighbors=3)

In [113]:
model_v2.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=3)

El modelo KNN, obtiene un accuracy del 76%, siendo el peor hasta la fecha

In [119]:
# Creamos un diccionario para establecer los labels en la variable y (necesario en el algoritmo XGBoost)
label_mapping = {
    'Insufficient_Weight': 0,
    'Normal_Weight': 1,
    'Obesity_Type_I': 2,
    'Obesity_Type_II': 3,
    'Obesity_Type_III': 4,
    'Overweight_Level_I': 5,
    'Overweight_Level_II': 6
}

In [120]:
# Aplicar el mapeo a las etiquetas de entrenamiento
y_train_encoded = y_train.map(label_mapping)

# Aplicar el mapeo a las etiquetas de validacion
y_val_encoded = y_val.map(label_mapping)

In [230]:
# Algoritmo XGBoost
model_v3 = XGBClassifier(n_estimators=200, max_depth=2, learning_rate=0.5, objective='multi:softmax')

In [231]:
model_v3.fit(X_train, y_train_encoded)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.5, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=2, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=200, n_jobs=None,
              num_parallel_tree=None, objective='multi:softmax', ...)

In [232]:
y_pred = model_v3.predict(X_val)

In [234]:
# Metrica accuracy score 
print('El modelo alcanza:',round(accuracy_score(y_val_encoded, y_pred), 3)*100,'% de precision')

El modelo alcanza: 90.3 % de precision


Como podemos ver, el ultimo algoritmo XGBoost, ha sido el mejor con diferencia, alcanzando una precision del 90%

In [240]:
# Funcion que da el valor categorico a cada clave del diccionario labels (necesario para volver a tener los valores como texto)
def label_to_text (pred, label_mapping):
    pred_copy = pred

    # Obtener las etiquetas originales en el mismo orden que los valores numéricos
    original_labels = [key for key, value in sorted(label_mapping.items(), key=lambda x: x[1])]

    # Mapear los valores numéricos a las etiquetas originales
    text_predictions = [original_labels[value] for value in pred_copy]
    return text_predictions

In [241]:
y_pred_text = label_to_text(y_pred, label_mapping)

Predecir con el conjunto de prueba y el algoritmo mas alto (XGBoost)

In [243]:
df_prueba = pd.read_csv('test.csv')
df_prueba

,id,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS
0,20758,Male,26.899886,1.848294,120.644178,yes,yes,2.938616,3.000000,Sometimes,no,2.825629,no,0.855400,0.000000,Sometimes,Public_Transportation
1,20759,Female,21.000000,1.600000,66.000000,yes,yes,2.000000,1.000000,Sometimes,no,3.000000,no,1.000000,0.000000,Sometimes,Public_Transportation
2,20760,Female,26.000000,1.643355,111.600553,yes,yes,3.000000,3.000000,Sometimes,no,2.621877,no,0.000000,0.250502,Sometimes,Public_Transportation
3,20761,Male,20.979254,1.553127,103.669116,yes,yes,2.000000,2.977909,Sometimes,no,2.786417,no,0.094851,0.000000,Sometimes,Public_Transportation
4,20762,Female,26.000000,1.627396,104.835346,yes,yes,3.000000,3.000000,Sometimes,no,2.653531,no,0.000000,0.741069,Sometimes,Public_Transportation
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13835,34593,Male,23.327836,1.721384,78.030383,yes,no,2.813234,3.000000,Sometimes,no,1.000000,no,0.807076,0.778632,Sometimes,Public_Transportation
13836,34594,Female,29.000000,1.590000,62.000000,no,yes,3.000000,3.000000,Sometimes,no,2.000000,no,0.000000,0.000000,Sometimes,Public_Transportation
13837,34595,Female,22.935612,1.585547,44.376637,no,yes,3.000000,2.273740,Frequently,no,2.000000,no,1.949840,1.000000,Sometimes,Public_Transportation
13838,34596,Male,21.000000,1.620000,53.000000,yes,yes,2.000000,3.000000,Sometimes,no,2.000000,no,3.000000,2.000000,no,Public_Transportation


In [244]:
# Guardamos id
id_ = df_prueba['id']

In [245]:
# Eliminamos id
df_prueba.drop('id', axis=1, inplace=True)

In [247]:
# Casteamos a entero
df_prueba = feature_float_int(df_prueba)
df_prueba

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS
0,Male,26,1.848294,120.644178,yes,yes,2,3,Sometimes,no,2,no,0,0,Sometimes,Public_Transportation
1,Female,21,1.600000,66.000000,yes,yes,2,1,Sometimes,no,3,no,1,0,Sometimes,Public_Transportation
2,Female,26,1.643355,111.600553,yes,yes,3,3,Sometimes,no,2,no,0,0,Sometimes,Public_Transportation
3,Male,20,1.553127,103.669116,yes,yes,2,2,Sometimes,no,2,no,0,0,Sometimes,Public_Transportation
4,Female,26,1.627396,104.835346,yes,yes,3,3,Sometimes,no,2,no,0,0,Sometimes,Public_Transportation
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13835,Male,23,1.721384,78.030383,yes,no,2,3,Sometimes,no,1,no,0,0,Sometimes,Public_Transportation
13836,Female,29,1.590000,62.000000,no,yes,3,3,Sometimes,no,2,no,0,0,Sometimes,Public_Transportation
13837,Female,22,1.585547,44.376637,no,yes,3,2,Frequently,no,2,no,1,1,Sometimes,Public_Transportation
13838,Male,21,1.620000,53.000000,yes,yes,2,3,Sometimes,no,2,no,3,2,no,Public_Transportation


In [248]:
# Codificamos categoricas
X_categorical_prueba = dummies_variable (df_prueba)
X_categorical_prueba

,Gender_Female,Gender_Male,family_history_with_overweight_no,family_history_with_overweight_yes,FAVC_no,FAVC_yes,CAEC_Always,CAEC_Frequently,CAEC_Sometimes,CAEC_no,...,SCC_yes,CALC_Always,CALC_Frequently,CALC_Sometimes,CALC_no,MTRANS_Automobile,MTRANS_Bike,MTRANS_Motorbike,MTRANS_Public_Transportation,MTRANS_Walking
0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13835,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
13836,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
13837,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
13838,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0


In [249]:
#Normalizamos numericas
X_numerical_prueba = normalizer_numerical (df_prueba)
X_numerical_prueba

,Age,Height,Weight,FCVC,NCP,CH2O,FAF,TUE
0,0.402742,1.682789,1.273786,-0.511426,0.454224,0.377939,-0.928250,-0.677774
1,-0.468051,-1.114658,-0.818988,-0.511426,-2.152642,2.108234,0.274088,-0.677774
2,0.402742,-0.626191,0.927432,1.201164,0.454224,0.377939,-0.928250,-0.677774
3,-0.642210,-1.642760,0.623672,-0.511426,-0.849209,0.377939,-0.928250,-0.677774
4,0.402742,-0.805996,0.668336,1.201164,0.454224,0.377939,-0.928250,-0.677774
...,...,...,...,...,...,...,...,...
13835,-0.119734,0.252936,-0.358246,-0.511426,0.454224,-1.352355,-0.928250,-0.677774
13836,0.925218,-1.227324,-0.972181,1.201164,0.454224,0.377939,-0.928250,-0.677774
13837,-0.293893,-1.277495,-1.647125,1.201164,-0.849209,0.377939,0.274088,0.996997
13838,-0.468051,-0.889324,-1.316865,-0.511426,0.454224,0.377939,2.678763,2.671768


In [250]:
# Concatenamos
df_prueba = concat_types(X_categorical_prueba,X_numerical_prueba)
df_prueba.head()

,Gender_Female,Gender_Male,family_history_with_overweight_no,family_history_with_overweight_yes,FAVC_no,FAVC_yes,CAEC_Always,CAEC_Frequently,CAEC_Sometimes,CAEC_no,...,MTRANS_Public_Transportation,MTRANS_Walking,Age,Height,Weight,FCVC,NCP,CH2O,FAF,TUE
0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.402742,1.682789,1.273786,-0.511426,0.454224,0.377939,-0.928250,-0.677774
1,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,...,1.0,0.0,-0.468051,-1.114658,-0.818988,-0.511426,-2.152642,2.108234,0.274088,-0.677774
2,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.402742,-0.626191,0.927432,1.201164,0.454224,0.377939,-0.928250,-0.677774
3,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,...,1.0,0.0,-0.642210,-1.642760,0.623672,-0.511426,-0.849209,0.377939,-0.928250,-0.677774
4,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.402742,-0.805996,0.668336,1.201164,0.454224,0.377939,-0.928250,-0.677774


In [251]:
df_prueba.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13840 entries, 0 to 13839
Data columns (total 31 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   Gender_Female                       13840 non-null  float64
 1   Gender_Male                         13840 non-null  float64
 2   family_history_with_overweight_no   13840 non-null  float64
 3   family_history_with_overweight_yes  13840 non-null  float64
 4   FAVC_no                             13840 non-null  float64
 5   FAVC_yes                            13840 non-null  float64
 6   CAEC_Always                         13840 non-null  float64
 7   CAEC_Frequently                     13840 non-null  float64
 8   CAEC_Sometimes                      13840 non-null  float64
 9   CAEC_no                             13840 non-null  float64
 10  SMOKE_no                            13840 non-null  float64
 11  SMOKE_yes                           13840

In [252]:
# Eliminamos la columna 'CALC_Always' debido a que no esta entrenada en el conjunto de entrenamiento.
df_prueba.drop('CALC_Always', axis=1, inplace=True)

In [253]:
# Prediccion del modelo entrenado en el de prueba
prediction_ = model_v3.predict(df_prueba)

In [255]:
# Convertimos a texto cada valor, dado el diccionario creado anteriormente
prediction_text = label_to_text(prediction_, label_mapping)

In [257]:
# Creamos el dataset de envio con la estructura exigida
submission = {'id': id_, 'NObeyesdad':prediction_text}

In [258]:
submit_v1 = pd.DataFrame(submission)
submit_v1

,id,NObeyesdad
0,20758,Obesity_Type_II
1,20759,Overweight_Level_I
2,20760,Obesity_Type_III
3,20761,Obesity_Type_I
4,20762,Obesity_Type_III
...,...,...
13835,34593,Overweight_Level_II
13836,34594,Normal_Weight
13837,34595,Insufficient_Weight
13838,34596,Normal_Weight


In [259]:
#Guardamos
submit_v1.to_csv('submitV1.csv', index=False)